In [44]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd

### Reading in the datasets
Datasets include either a glacial erosion rate compilation and a non glacial erosion rate compilation. 

In [46]:
data_glacial = pd.read_csv ("glacial_erosion_Earth.tsv", sep = '\t')
data_glacial.head()

,Compilation,Reference,Reference DOI,Locality,Glacier,Region,Latitude,Longitude,Elevation (m),Type,...,Area (km2),Time interval (yr),Sediment flux (km3/yr),Exhumation (m),Exhumation min (m),Exhumation max (m),Erosion rate (mm/yr),Erosion rate min,Erosion rate max,Comments
0,"Amantov et al., 2011","Boulton et al., 1991",10.1007/978-3-642-17220-5_3,UK/Scotland,unnamed,British Isles,53.34430,-1.87818,NaN,Continental,...,NaN,1.000000e+05,NaN,NaN,NaN,NaN,1.0,NaN,NaN,"Boulton GS, Peacock JD, Sutherland DG (1991) Q..."
1,"Amantov et al., 2011","Clayton, 1996",10.1007/978-3-642-17220-5_3,UK/Britain,unnamed,British Isles,53.34430,-1.87818,NaN,Continental,...,272000.0,3.000000e+05,NaN,NaN,NaN,NaN,17.0,11.0,23.0,Assumes all erosion during glacials (15% of la...
2,"Amantov et al., 2011","Faleide et al., 1996",10.1007/978-3-642-17220-5_3,Barents Sea-Svalbard continental margin,unnamed,Fennoscandia,67.18026,25.40585,NaN,High-latitude,...,NaN,2.500000e+06,NaN,NaN,NaN,NaN,0.6,0.1,1.1,"Faleide JI, Solheim A, Fiedler A, Hjelstuen BO..."
3,"Amantov et al., 2011","Solheim et al., 1996",10.1007/978-3-642-17220-5_3,Norwegian margin,unnamed,Fennoscandia,62.28011,8.68988,NaN,High-latitude,...,NaN,2.500000e+06,NaN,NaN,NaN,NaN,0.4,0.2,0.6,"Solheim A, Riis F, Elverhøi A, Faleide JI, Jen..."
4,"Anderson, 2005","Kraal, 2001; Anderson et al., 2003",10.1016/j.chemgeo.2003.01.001,"Wrangell Mountains, Alaska",Kennicott Glacier,North American Cordillera,61.59991,-143.06951,NaN,Alpine,...,420.0,1.666670e-01,NaN,NaN,NaN,NaN,0.8,NaN,NaN,Summer 1999


In [47]:
data_fluvial = pd.read_csv ("nonglacial_erosion_Earth.tsv", sep = '\t')
data_fluvial.head()

,Compilation,Reference,Reference DOI,Locality,Basin,Place,Region,Sample ID,Latitude,Longitude,...,Area (km2),Time interval (yr),Sediment flux (km3/yr),Exhumation (m),Exhumation min (m),Exhumation max (m),Erosion rate (mm/yr),Erosion rate min,Erosion rate max,Comments
0,"Burbank et al., 2000","Amato et al., 1999",10.1016/S0012-821X(99)00161-2,Zermatt-Saas ophiolite,NaN,European Alps,Europe,NaN,46.02065,7.74914,...,NaN,20000000.0,NaN,6.0,NaN,NaN,3.0,NaN,NaN,Possibly glacial or partly glacial?
1,"Burbank et al., 2000","Amato et al., 1999",10.1016/S0012-821X(99)00161-2,Zermatt-Saas,NaN,European Alps,Europe,NaN,46.02065,7.74914,...,NaN,4750000.0,NaN,65.0,NaN,NaN,18.0,10.0,26.0,Possibly glacial or partly glacial?
2,"Burbank et al., 2000","Brandon et al., 1998",10.1130/0016-7606(1998)110%3C0985:LCEOTC%3E2.3...,"Ultrahigh-P rocks, Olympic Range, Washington",NaN,USA,North America,NaN,47.75000,-123.50000,...,NaN,11000000.0,NaN,9.0,NaN,NaN,1.2,NaN,NaN,Possibly glacial or partly glacial?
3,"Burbank et al., 2000","Foster and John, 1999",10.1144/GSL.SP.1999.154.01.16,"Chemehuevi Mountains, Basin and Range",NaN,USA,North America,NaN,34.62437,-114.52653,...,NaN,4000000.0,NaN,12.5,10.0,15.0,3.0,2.0,4.0,NaN
4,"Burbank et al., 2000","Hacker et al., 2003",10.1016/S0024-4937(03)00092-6,Kokchetav Massif,NaN,Kazakhstan,Central Asia,NaN,53.08000,70.18833,...,NaN,7000000.0,NaN,160.0,NaN,NaN,20.0,NaN,NaN,NaN


In [48]:
data_glacial['type'] = 'Glacial'
data_fluvial['type'] = 'Fluvial'

### Preprocessing Data

In [51]:
def preprocess_data(dff, dfg):
    
    world = pd.DataFrame() # initializing a dataframe
    
    # latitudes
    flat = np.array(dff['Latitude'])
    glat = np.array(dfg['Latitude'])
    lats = np.append(flat, glat)
    world['lat'] = lats
    
    # longitudes
    flon = np.array(dff['Longitude'])
    glon = np.array(dfg['Longitude'])
    longs = np.append(flon, glon)
    world['long'] = longs
    
#     # region
    freg = np.array(dff['Region'])
    greg = np.array(dfg['Region'])
    region = np.append(freg, greg)
    world['region'] = region
    
#     # locality
    floc = np.array(dff['Locality'])
    gloc = np.array(dfg['Locality'])
    locs = np.append(floc, gloc)
    world['locality'] = locs
    
    # type
    ftype = np.array(dff['type'])
    gtype = np.array(dfg['type'])
    types = np.append(ftype, gtype)
    world['type'] = types
    
    # glacier/basin
    basin = np.array(dff['Basin'])
    glacier = np.array(dfg['Glacier'])
    names = np.append(basin, glacier)
    world['names'] = names
    
    return world

In [52]:
world_df = preprocess_data(data_fluvial, data_glacial)

### Map Function

In [53]:
def generate_map(world, **kwargs):
    
    fig = px.scatter_geo(world,
                        lat='lat',
                        lon='long',
                        hover_data=['locality','region'],
                        hover_name='names',
                        color='type',
                        color_discrete_sequence=['#FF7F0E','#1F77B4'],
                        **kwargs
                        )
    fig.update(layout={"title": {"x": 0.85, "y": 0.97}})
    fig.update_layout(margin={"r":0,"t":30,"l":0,"b":10})
    
    fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.50,
    xanchor="left",
    x=0.01,
), legend_font_size=15, legend_title=' ')
    
    return fig

In [54]:
proj = 'equirectangular'
# proj = 'orthographic'
# proj = 'robinson'
# proj = 'mollweide'
# proj = 'winkel tripel'
# proj = 'sinusoidal'

fig = generate_map(world_df, projection=proj, scope='world', opacity=0.3, template='ggplot2') 
fig.show()